In [ ]:
# enable connection to google drive file system
from google.colab import drive
drive.mount("/content/drive")
# import required modules here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import preprocessing
from scipy.stats import norm
import statsmodels.api as sm

In [2]:
# analysis of dataset before basic engineering
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/bout_statistics.csv"
df = pd.read_csv(path)
print(len(df.index))
print(len(df.columns))
print(df.columns)

6987
57
Index(['card_date', 'card_name', 'red_name', 'blue_name', 'winner_name',
       'loser_name', 'winner_color', 'loser_color', 'match_result',
       'num_rounds', 'time', 'method', 'time_format', 'referee_name',
       'division', 'sig_str_total_red', 'sig_str_landed_red',
       'sig_str_total_blue', 'sig_str_landed_blue', 'total_str_total_red',
       'total_str_landed_red', 'total_str_total_blue', 'total_str_landed_blue',
       'td_total_red', 'td_landed_red', 'td_total_blue', 'td_landed_blue',
       'sub_att_red', 'sub_att_blue', 'rev_red', 'rev_blue', 'ctrl_red',
       'ctrl_blue', 'head_str_total_red', 'head_str_landed_red',
       'head_str_total_blue', 'head_str_landed_blue', 'body_str_total_red',
       'body_str_landed_red', 'body_str_total_blue', 'body_str_landed_blue',
       'leg_str_total_red', 'leg_str_landed_red', 'leg_str_total_blue',
       'leg_str_landed_blue', 'distance_str_total_red',
       'distance_str_landed_red', 'distance_str_total_blue',
       'd

In [3]:
# note: UFC rounds are each 5 minutes long
# SECTION A: BASIC ENGINEERING OF DATASET
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/bout_statistics.csv"
df = pd.read_csv(path)
# removing columns that are not necessary
df = df.drop(columns=['card_name', 'referee_name', 'winner_name', 'loser_name'])
df['card_date'] = pd.to_datetime(df['card_date'])
# take bouts with the 1 Rnd + OT format out of consideration
df = df.drop(df[df['time_format'] == '1'].index)
df = df.drop(df[df['time_format'] == 'No'].index)
df = df.astype({'time_format':'int'})
# take bouts that ended as no contests out of the dataset as these are anomalies
df = df.drop(df[df['match_result'] == 'NC'].index)
# take bouts that ended in disqualifications as these are anomalies
df = df.drop(df[df['method'] == 'DQ'].index)
# drop rows that are missing statistical values (for the earliest ufc matches)
df = df.drop(df[df['ctrl_red'] == '--'].index)
df = df.drop(df[df['ctrl_blue'] == "--"].index)
# engineer: control time (seconds), max bout duration (seconds), actual bout duration (seconds)
df['ctrl_red'] = df['ctrl_red'].apply(lambda x: 60 * int(x.split(":")[0]) + int(x.split(":")[1]))
df['ctrl_blue'] = df['ctrl_blue'].apply(lambda x: 60 * int(x.split(":")[0]) + int(x.split(":")[1]))
df['max_bout_duration'] = df['time_format'].apply(lambda x: 60 * 5 * x) 
def calc_actual_bout_duration(row): 
  min = int(row['time'].split(":")[0])
  sec = int(row['time'].split(":")[1])
  return 60 * 5 * (row['num_rounds'] - 1) + 60 * min + sec
df['actual_bout_duration'] = df.apply(lambda row: calc_actual_bout_duration(row), axis=1)
df = df.drop(columns=['time', 'num_rounds', 'time_format'])
# engineer: red_win - Yes/No and blue_win - Yes/No. Note: draw means that both red and blue don't win
def return_win(row, red):
  if red == True:
    if row['winner_color'] == 'red':
      return 'Yes'
    else:
      return 'No'
  else:
    if row['winner_color'] == 'blue':
      return 'Yes'
    else:
      return 'No'
df['red_win'] = df.apply(lambda row: return_win(row, red=True), axis=1)
df['blue_win'] = df.apply(lambda row: return_win(row, red=False), axis=1)
df['red_win'] = df['red_win'].astype('category')
df['blue_win'] = df['blue_win'].astype('category')
df = df.drop(columns=['winner_color', 'loser_color', 'match_result'])
# convert method and division into categorical types
df['method'] = df['method'].astype('category')
df['division'] = df['division'].astype('category')

In [ ]:
# export basic engineered dataset as csv file
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/basic_eng_statistics.csv"
df.to_csv(path, index=False)

In [4]:
# PRE SECTION C: OBSERVE PERFORMANCE WHEN CUTTING SIZE OF DATASET
CUT_PROP = 1.0
cut_num = int(len(df) * CUT_PROP)
df = df.iloc[:cut_num]

In [ ]:
# SECTION B: CONDUCT PRELIM VISUALIZATION OF BASIC ENGINEERED DATASET
pd.set_option("display.max_rows", None)
with pd.option_context('display.max_columns', 100):
  print(df.info(), "\n")
  print(df.describe(include='all', datetime_is_numeric=True), "\n")
  print(df['method'].value_counts(), "\n")
  print(df['division'].value_counts(), "\n")
  print(df.columns)

In [6]:
# SECTION C: CUSTOM ENGINEERING OF DATASET (trial 1)
# see how model performs without the inclusion of specific details on significant strikes
# (intuition is that including these features will overfit the model at the expense of other features)
df1 = df.drop(columns=['head_str_total_red', 'head_str_landed_red', 'head_str_total_blue', 'head_str_landed_blue', 
                      'body_str_total_red', 'body_str_landed_red', 'body_str_total_blue', 'body_str_landed_blue', 
                      'leg_str_total_red', 'leg_str_landed_red', 'leg_str_total_blue', 'leg_str_landed_blue',
                      'distance_str_total_red', 'distance_str_landed_red', 'distance_str_total_blue', 'distance_str_landed_blue',
                      'clinch_str_total_red', 'clinch_str_landed_red', 'clinch_str_total_blue', 'clinch_str_landed_blue',
                      'ground_str_total_red', 'ground_str_landed_red', 'ground_str_total_blue', 'ground_str_landed_blue'])
# see how model performs when ignoring fighting division
df1 = df1.drop(columns=['division'])
# transform attack counts to attack rates and success ratios
# (attack rates normalizes raw attack statistics with regards to the time spent fighting)
# (provide a 100% success ratio if an attack of a specified type was never made)
df1['all_str_hit_ratio_red'] = df1.apply(lambda row: 1 if row.total_str_total_red == 0 else row.total_str_landed_red / row.total_str_total_red, axis=1)
df1['all_str_attempt_rate_red'] = df1.apply(lambda row: (row.total_str_total_red / row.actual_bout_duration) * 60, axis=1)
df1['sig_str_hit_ratio_red'] =  df1.apply(lambda row: 1 if row.sig_str_total_red == 0 else row.sig_str_landed_red / row.sig_str_total_red, axis=1)
df1['sig_str_attempt_rate_red'] = df1.apply(lambda row: (row.sig_str_total_red / row.actual_bout_duration) * 60, axis=1)
df1['td_success_ratio_red'] = df1.apply(lambda row: 1 if row.td_total_red == 0 else row.td_landed_red / row.td_total_red, axis=1)
df1['td_attempt_rate_red'] = df1.apply(lambda row: (row.td_total_red / row.actual_bout_duration) * 60, axis=1)
df1['sub_attempt_rate_red'] = df1.apply(lambda row: (row.sub_att_red / row.actual_bout_duration) * 60, axis=1)
df1['all_str_hit_ratio_blue'] = df1.apply(lambda row: 1 if row.total_str_total_blue == 0 else row.total_str_landed_blue / row.total_str_total_blue, axis=1)
df1['all_str_attempt_rate_blue'] = df1.apply(lambda row: (row.total_str_total_blue / row.actual_bout_duration) * 60, axis=1)
df1['sig_str_hit_ratio_blue'] =  df1.apply(lambda row: 1 if row.sig_str_total_blue == 0 else row.sig_str_landed_blue / row.sig_str_total_blue, axis=1)
df1['sig_str_attempt_rate_blue'] = df1.apply(lambda row: (row.sig_str_total_blue / row.actual_bout_duration) * 60, axis=1)
df1['td_success_ratio_blue'] = df1.apply(lambda row: 1 if row.td_total_blue == 0 else row.td_landed_blue / row.td_total_blue, axis=1)
df1['td_attempt_rate_blue'] = df1.apply(lambda row: (row.td_total_blue / row.actual_bout_duration) * 60, axis=1)
df1['sub_attempt_rate_blue'] = df1.apply(lambda row: (row.sub_att_blue / row.actual_bout_duration) * 60, axis=1)
# NEW CUSTOM ENGINEERED STATISTICS !!!
df1['all_str_hit_rate_red'] = df1.apply(lambda row: (row.total_str_landed_red / row.actual_bout_duration) * 60, axis=1)
df1['sig_str_hit_rate_red'] = df1.apply(lambda row: (row.sig_str_landed_red / row.actual_bout_duration) * 60, axis=1)
df1['all_str_hit_rate_blue'] = df1.apply(lambda row: (row.total_str_landed_blue / row.actual_bout_duration) * 60, axis=1)
df1['sig_str_hit_rate_blue'] = df1.apply(lambda row: (row.sig_str_landed_blue / row.actual_bout_duration) * 60, axis=1)
# transform time spent fighting and control time to ratios
# (ratio normalizes raw time statistics with regards to different round formats)
df1['ctrl_time_ratio_red'] = df1.apply(lambda row: row.ctrl_red / row.actual_bout_duration, axis=1)
df1['ctrl_time_ratio_blue'] = df1.apply(lambda row: row.ctrl_blue / row.actual_bout_duration, axis=1)
df1['prop_match_completed'] = df1.apply(lambda row: row.actual_bout_duration / row.max_bout_duration, axis=1)
# aggregate win method into groups and one hot encode the win method
df1['method'].replace(to_replace=['KO/TKO', 'TKO - Doctor\'s Stoppage'], value=['KO/TKO', 'KO/TKO'], inplace=True)
df1['method'].replace(to_replace=['Decision - Split', 'Decision - Majority'], value=['NU_Decision', 'NU_Decision'], inplace=True)
df1['method'].replace(to_replace=['Decision - Unanimous'], value=['U_Decision'], inplace=True)
# drop remaining columns that are not needed
df1 = df1.drop(columns=['sig_str_total_red', 'sig_str_landed_red', 'sig_str_total_blue', 'sig_str_landed_blue',
                  'total_str_total_red', 'total_str_landed_red', 'total_str_total_blue', 'total_str_landed_blue', 
                  'td_total_red', 'td_landed_red', 'td_total_blue', 'td_landed_blue', 
                  'sub_att_red', 'sub_att_blue', 'rev_red', 'rev_blue', 'ctrl_red', 'ctrl_blue', 
                  'max_bout_duration', 'actual_bout_duration'])

In [ ]:
# SECTION D: CONDUCT VISUALIZATION AND UNDERSTANDING OF CUSTOM ENGINEERED DATASET (trial 1)
pd.set_option("display.max_rows", None)
with pd.option_context('display.max_columns', 100):
  print(df1.info(), "\n")
  # print(df1.describe(include='all', datetime_is_numeric=True), "\n")
  pass
# make a histogram of the number of bouts each fighter has participated in
ax = pd.concat([df1['red_name'], df1['blue_name']]).value_counts().plot.hist(alpha=0.5, bins=30)
ax.set_title("Number Of Bouts Each Fighter Has Fought In")
plt.xlabel("num bouts")
plt.ylabel("num fighters")
plt.show()
ax = pd.concat([df1['red_name'], df1['blue_name']]).value_counts().plot.box(vert=False)
ax.set_title("Number Of Bouts Each Fighter Has Fought In")
plt.xlabel("num bouts")
plt.show()
# make a histogram of red win vs blue win
ax = df1['red_win'].value_counts().plot.pie()
ax.set_title("Percent Matches Won By Red")
plt.show()
print(f"{round((df1['red_win'].value_counts()[0] / df1['red_win'].value_counts().sum()) * 100)}%")
ax = df1['blue_win'].value_counts().plot.pie()
ax.set_title("Percent Matches Won By Blue")
plt.show()
print(f"{round((df1['blue_win'].value_counts()[1] / df1['red_win'].value_counts().sum()) * 100)}%")
# make a bar chart of the method by which the bout was ended
ax = df1['method'].value_counts().plot.bar()
ax.set_title("Bout End Method")
plt.xlabel("method")
plt.ylabel("num bouts")
plt.show()
# make histograms of red's attack success ratio
ax = df1['all_str_hit_ratio_red'].plot.hist()
ax.set_title("All Strikes Hit Ratio Of Red")
plt.xlabel("hit ratio")
plt.ylabel("num bouts")
plt.show()
ax = df1['sig_str_hit_ratio_red'].plot.hist()
ax.set_title("Significant Strikes Hit Ratio Of Red")
plt.xlabel("hit ratio")
plt.ylabel("num bouts")
plt.show()
ax = df1['td_success_ratio_red'].plot.hist()
ax.set_title("Takedown Success Ratio Of Red")
plt.xlabel("success ratio")
plt.ylabel("num bouts")
plt.show()
# make histograms of red's attack rate
ax = df1['all_str_attempt_rate_red'].plot.hist()
ax.set_title("All Strikes Attempt Rate Of Red")
plt.xlabel("rate")
plt.ylabel("num bouts")
plt.show()
ax = df1['sig_str_attempt_rate_red'].plot.hist()
ax.set_title("Significant Strikes Attempt Rate Of Red")
plt.xlabel("rate")
plt.ylabel("num bouts")
plt.show()
ax = df1['td_attempt_rate_red'].plot.hist()
ax.set_title("Takedown Attempt Rate Of Red")
plt.xlabel("rate")
plt.ylabel("num bouts")
plt.show()
ax = df1['sub_attempt_rate_red'].plot.hist()
ax.set_title("Submission Attempt Rate Of Red")
plt.xlabel("rate")
plt.ylabel("num bouts")
plt.show()
# make histogram of red's control time ratio
ax = df1['ctrl_time_ratio_red'].plot.hist()
ax.set_title("Control Time Ratio Of Red")
plt.xlabel("time ratio")
plt.ylabel("num bouts")
plt.show()
# make histogram of match completion ratio
ax = df1['prop_match_completed'].plot.hist()
ax.set_title("Proportion Of Match Completed")
plt.xlabel("time ratio")
plt.ylabel("num bouts")
plt.show()
# make a heatmap of red win vs match end method
table = pd.crosstab(df['red_win'], df['method'])
sns.heatmap(table, annot=True, cmap='coolwarm')
plt.title("Red Win vs Match End Method")
plt.show()
# make a heatmap of red win vs all other quantitative statistics
table = pd.pivot_table(df1, values=[
    'all_str_hit_ratio_red', 'sig_str_hit_ratio_red', 'td_success_ratio_red', 'ctrl_time_ratio_red',
    'all_str_hit_ratio_blue', 'sig_str_hit_ratio_blue', 'td_success_ratio_blue', 'ctrl_time_ratio_blue',
    'prop_match_completed'], index=['red_win'], aggfunc='mean')
plt.figure(figsize=(10, 3))
sns.heatmap(table, annot=True, cmap='coolwarm')
plt.title("Red Win vs Attack Success Ratio")
plt.show()
table = pd.pivot_table(df1, values=[
    'all_str_attempt_rate_red', 'sig_str_attempt_rate_red', 'td_attempt_rate_red', 'sub_attempt_rate_red',
    'all_str_attempt_rate_blue', 'sig_str_attempt_rate_blue', 'td_attempt_rate_blue', 'sub_attempt_rate_blue'
    ], index=['red_win'], aggfunc='mean')
plt.figure(figsize=(10, 3))
sns.heatmap(table, annot=True, cmap='coolwarm')
plt.title("Red Win vs Proportion Statistics")
plt.show()
# make a heatmap between match end method vs all other quantitative statistics
table = pd.pivot_table(df1, values=[
    'all_str_hit_ratio_red', 'sig_str_hit_ratio_red', 'td_success_ratio_red', 'ctrl_time_ratio_red',
    'all_str_hit_ratio_blue', 'sig_str_hit_ratio_blue', 'td_success_ratio_blue', 'ctrl_time_ratio_blue',
    'prop_match_completed'], index=['method'], aggfunc='mean')
plt.figure(figsize=(10, 3))
sns.heatmap(table, annot=True, cmap='coolwarm')
plt.title("Match End Method vs Attack Success Ratio")
plt.show()
table = pd.pivot_table(df1, values=[
    'all_str_attempt_rate_red', 'sig_str_attempt_rate_red', 'td_attempt_rate_red', 'sub_attempt_rate_red',
    'all_str_attempt_rate_blue', 'sig_str_attempt_rate_blue', 'td_attempt_rate_blue', 'sub_attempt_rate_blue'
    ], index=['method'], aggfunc='mean')
plt.figure(figsize=(10, 3))
sns.heatmap(table, annot=True, cmap='coolwarm')
plt.title("Match End Method vs Proportion Statistics")
plt.show()
# make a correlative heatmap within quantitative statistics
cm = df1[['all_str_hit_ratio_red', 'sig_str_hit_ratio_red', 'td_success_ratio_red', 'ctrl_time_ratio_red',
    'all_str_hit_ratio_blue', 'sig_str_hit_ratio_blue', 'td_success_ratio_blue', 'ctrl_time_ratio_blue',
    'all_str_attempt_rate_red', 'sig_str_attempt_rate_red', 'td_attempt_rate_red', 'sub_attempt_rate_red',
    'all_str_attempt_rate_blue', 'sig_str_attempt_rate_blue', 'td_attempt_rate_blue', 'sub_attempt_rate_blue',
    'prop_match_completed']].corr()
plt.figure(figsize=(15, 15))
sns.heatmap(cm, annot=True, cmap='coolwarm')
plt.title("Intercorrelation Between Quantitative Variables")
plt.show()

In [ ]:
df11 = df1.copy()
print(df11.columns)
# label encode red win and blue win
df11['red_win'] = df11['red_win'].cat.codes
df11['blue_win'] = df11['blue_win'].cat.codes
# one hot encode the match end method
df11 = pd.get_dummies(df11, columns=['method'])
# predict which variables will be the most useful for predicting match result
X = df11.drop(columns=['red_win', 'blue_win', 'card_date', 'red_name', 'blue_name'])
y = df11['red_win']
selector = SelectKBest(score_func=mutual_info_classif, k=10)
selector.fit_transform(X, y)
scores = selector.scores_
ranked_features = X.columns[np.argsort(-scores)]
print()
print("Feature ranking:")
for i, feature in enumerate(ranked_features):
    print("%d. %s (%f)" % (i+1, feature, scores[X.columns.get_loc(feature)]))

In [ ]:
# export custom engineered dataset as csv file
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/custom_eng_statistics_1.csv"
df11.to_csv(path, index=False)

In [ ]:
# SECTION E: TRAIN MODEL AND TEST USING PROPHETIC DATA
# metrics when using 2 grappling only features
train_df11, test_df11 = train_test_split(df11, test_size=0.2, random_state=50)
X_train = train_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue']]
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue']]
y_test = test_df11['red_win']
model = LogisticRegression(random_state=50)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 custom control only)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking only features
train_df11, test_df11 = train_test_split(df11, test_size=0.2, random_state=50)
X_train = train_df11[['all_str_attempt_rate_blue', 'all_str_attempt_rate_red', 
                      'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_train = train_df11['red_win']
X_test = test_df11[['all_str_attempt_rate_blue', 'all_str_attempt_rate_red',
                    'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_test = test_df11['red_win']
model = LogisticRegression(random_state=50)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 custom striking only)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 success only features
train_df11, test_df11 = train_test_split(df11, test_size=0.2, random_state=50)
X_train = train_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue',  
                      'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue',
                    'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_test = test_df11['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 custom success only)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 6 custom selected features
train_df11, test_df11 = train_test_split(df11, test_size=0.2, random_state=50)
X_train = train_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_blue', 
        'all_str_attempt_rate_red', 'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_blue', 
        'all_str_attempt_rate_red', 'all_str_hit_ratio_blue', 'all_str_hit_ratio_red']]
y_test = test_df11['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 via custom selection)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
sm_model = sm.Logit(y_train, sm.add_constant(X_train)).fit(disp=0)
print(sm_model.pvalues)
print()
# metrics when selecting top 6 features from feature selection
train_df11, test_df11 = train_test_split(df11, test_size=0.2)
X_train = train_df11[['ctrl_time_ratio_red', 'all_str_attempt_rate_blue', 'sig_str_hit_ratio_blue', 
        'sig_str_attempt_rate_blue', 'sig_str_hit_ratio_red', 'ctrl_time_ratio_blue']]
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'all_str_attempt_rate_blue', 'sig_str_hit_ratio_blue', 
        'sig_str_attempt_rate_blue', 'sig_str_hit_ratio_red', 'ctrl_time_ratio_blue']]
y_test = test_df11['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 via fs)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 10 custom selected features
train_df11, test_df11 = train_test_split(df11, test_size=0.2)
X_train = train_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_blue', 
        'all_str_attempt_rate_red', 'all_str_hit_ratio_blue', 'all_str_hit_ratio_red',
        'sub_attempt_rate_red', 'sub_attempt_rate_blue', 'td_success_ratio_red', 'td_success_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_blue', 
        'all_str_attempt_rate_red', 'all_str_hit_ratio_blue', 'all_str_hit_ratio_red',
        'sub_attempt_rate_red', 'sub_attempt_rate_blue', 'td_success_ratio_red', 'td_success_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df11['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (10 custom): ")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when selecting top 10 features from feature selection
train_df11, test_df11 = train_test_split(df11, test_size=0.2)
X_train = train_df11[['ctrl_time_ratio_red', 'all_str_attempt_rate_blue', 'sig_str_hit_ratio_blue', 
        'sig_str_attempt_rate_blue', 'sig_str_hit_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_red',
        'all_str_hit_ratio_blue', 'all_str_hit_ratio_red', 'sub_attempt_rate_red']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df11['red_win']
X_test = test_df11[['ctrl_time_ratio_red', 'all_str_attempt_rate_blue', 'sig_str_hit_ratio_blue', 
        'sig_str_attempt_rate_blue', 'sig_str_hit_ratio_red', 'ctrl_time_ratio_blue', 'all_str_attempt_rate_red',
        'all_str_hit_ratio_blue', 'all_str_hit_ratio_red', 'sub_attempt_rate_red']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df11['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (10 via fs)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

In [8]:
# 2 MINUTES TO RUN
# SECTION F (part 2): ENGINEER DATASET OF PRIORS (trial 1 + via last fight imputation)
# !!! REVISED TO INCLUDE NEW DEFENSE STATISTICS !!!
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/custom_eng_statistics_1.csv"
new_df11 = pd.read_csv(path)
# remove prophetic information from imputation dataframe
df1i = new_df11.drop(columns=['all_str_hit_ratio_red', 'all_str_attempt_rate_red',
                          'sig_str_hit_ratio_red', 'sig_str_attempt_rate_red', 'td_success_ratio_red', 
                          'td_attempt_rate_red', 'sub_attempt_rate_red', 'all_str_hit_ratio_blue', 
                          'all_str_attempt_rate_blue', 'sig_str_hit_ratio_blue', 'sig_str_attempt_rate_blue',
                          'td_success_ratio_blue', 'td_attempt_rate_blue', 'sub_attempt_rate_blue', 
                          'ctrl_time_ratio_red', 'ctrl_time_ratio_blue', 'prop_match_completed', 
                          'method_NU_Decision', 'method_U_Decision', 'method_KO/TKO', 'method_Submission',
                          'all_str_hit_rate_red', 'sig_str_hit_rate_red', 'all_str_hit_rate_blue', 'sig_str_hit_rate_blue'])
# red fighter's features to be imputed
df1i['num_fights_red'] = np.zeros(len(new_df11))
df1i['prop_wins_red'] = np.zeros(len(new_df11))
df1i['imp_all_str_hit_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_all_str_attempt_rate_red'] = np.zeros(len(new_df11))
df1i['imp_sig_str_hit_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_sig_str_attempt_rate_red'] = np.zeros(len(new_df11))
df1i['imp_td_success_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_td_attempt_rate_red'] = np.zeros(len(new_df11))
df1i['imp_ctrl_time_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_prop_match_completed_red'] = np.zeros(len(new_df11))
df1i['imp_num_win_NU_Decision_red'] = np.zeros(len(new_df11))
df1i['imp_num_win_U_Decision_red'] = np.zeros(len(new_df11))
df1i['imp_num_win_KO/TKO_red'] = np.zeros(len(new_df11))
df1i['imp_num_win_Submission_red'] = np.zeros(len(new_df11))
# NEW RED FIGHTER's DEFENSE STAT FEATURES TO BE IMPUTED
df1i['imp_all_str_absorb_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_all_str_absorb_rate_red'] = np.zeros(len(new_df11))
df1i['imp_sig_str_absorb_ratio_red'] = np.zeros(len(new_df11))
df1i['imp_sig_str_absorb_rate_red'] = np.zeros(len(new_df11))
df1i['imp_ctrled_time_ratio_red'] = np.zeros(len(new_df11))
# blue fighter's features to be imputed
df1i['num_fights_blue'] = np.zeros(len(new_df11))
df1i['prop_wins_blue'] = np.zeros(len(new_df11))
df1i['imp_all_str_hit_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_all_str_attempt_rate_blue'] = np.zeros(len(new_df11))
df1i['imp_sig_str_hit_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_sig_str_attempt_rate_blue'] = np.zeros(len(new_df11))
df1i['imp_td_success_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_td_attempt_rate_blue'] = np.zeros(len(new_df11))
df1i['imp_ctrl_time_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_prop_match_completed_blue'] = np.zeros(len(new_df11))
df1i['imp_num_win_NU_Decision_blue'] = np.zeros(len(new_df11))
df1i['imp_num_win_U_Decision_blue'] = np.zeros(len(new_df11))
df1i['imp_num_win_KO/TKO_blue'] = np.zeros(len(new_df11))
df1i['imp_num_win_Submission_blue'] = np.zeros(len(new_df11))
# NEW BLUE FIGHTER' DEFENSE STAT FEATURES TO BE IMPUTED
df1i['imp_all_str_absorb_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_all_str_absorb_rate_blue'] = np.zeros(len(new_df11))
df1i['imp_sig_str_absorb_ratio_blue'] = np.zeros(len(new_df11))
df1i['imp_sig_str_absorb_rate_blue'] = np.zeros(len(new_df11))
df1i['imp_ctrled_time_ratio_blue'] = np.zeros(len(new_df11))
# iterate through all rows of df1i, extracting from df1: previous stats for red_name and for blue_name
for index, row in df1i.iterrows():
  # extract rows from prophetic dataframe relevant to red_name
  red_df = new_df11.loc[(new_df11['card_date'] < row['card_date']) & ((new_df11['red_name'] == row['red_name']) | (new_df11['blue_name'] == row['red_name']))]
  red_win_df = red_df.loc[((red_df['red_name'] == row['red_name']) & (red_df['red_win'] == 1)) | ((red_df['blue_name'] == row['red_name']) & (red_df['blue_win']) == 1)]
  # use extracted rows to impute mean features
  num_fights_red = red_df.shape[0]
  num_wins_red = red_win_df.shape[0]
  df1i.loc[index, 'num_fights_red'] = num_fights_red
  if num_fights_red > 0:
    imp_all_str_hit_ratio_red = 0
    imp_all_str_attempt_rate_red = 0
    imp_sig_str_hit_ratio_red = 0
    imp_sig_str_attempt_rate_red = 0
    imp_td_success_ratio_red = 0
    imp_td_attempt_rate_red = 0
    imp_ctrl_time_ratio_red = 0
    imp_prop_match_completed_red = 0
    # NEW RED FEATURES
    imp_all_str_absorb_ratio_red = 0
    imp_all_str_absorb_rate_red = 0
    imp_sig_str_absorb_ratio_red = 0
    imp_sig_str_absorb_rate_red = 0
    imp_ctrled_time_ratio_red = 0
    df1i.loc[index, 'prop_wins_red'] = num_wins_red / num_fights_red
    row2 = red_df.loc[red_df['card_date'] == red_df['card_date'].max()].iloc[0]
    if row2['red_name'] == row['red_name']:
      # FORGOT TO ADD THIS IN
      imp_prop_match_completed_red += row2['prop_match_completed']
      # END OF FORGOT TO ADD THIS IN 
      imp_all_str_hit_ratio_red += row2['all_str_hit_ratio_red']
      imp_all_str_attempt_rate_red += row2['all_str_attempt_rate_red']
      imp_sig_str_hit_ratio_red += row2['sig_str_hit_ratio_red']
      imp_sig_str_attempt_rate_red += row2['sig_str_attempt_rate_red']
      imp_td_success_ratio_red += row2['td_success_ratio_red']
      imp_td_attempt_rate_red += row2['td_attempt_rate_red']
      imp_ctrl_time_ratio_red += row2['ctrl_time_ratio_red']
      # NEW RED FEATURES
      imp_all_str_absorb_ratio_red = row2['all_str_hit_ratio_blue']
      imp_all_str_absorb_rate_red = row2['all_str_hit_rate_blue']
      imp_sig_str_absorb_ratio_red = row2['sig_str_hit_ratio_blue']
      imp_sig_str_absorb_rate_red = row2['sig_str_hit_rate_blue']
      imp_ctrled_time_ratio_red = row2['ctrl_time_ratio_blue']
    elif row2['blue_name'] == row['red_name']:
      imp_all_str_hit_ratio_red += row2['all_str_hit_ratio_blue']
      imp_all_str_attempt_rate_red += row2['all_str_attempt_rate_blue']
      imp_sig_str_hit_ratio_red += row2['sig_str_hit_ratio_blue']
      imp_sig_str_attempt_rate_red += row2['sig_str_attempt_rate_blue']
      imp_td_success_ratio_red += row2['td_success_ratio_blue']
      imp_td_attempt_rate_red += row2['td_attempt_rate_blue']
      imp_ctrl_time_ratio_red += row2['ctrl_time_ratio_blue']
      # NEW RED FEATURES
      imp_all_str_absorb_ratio_red = row2['all_str_hit_ratio_red']
      imp_all_str_absorb_rate_red = row2['all_str_hit_rate_red']
      imp_sig_str_absorb_ratio_red = row2['sig_str_hit_ratio_red']
      imp_sig_str_absorb_rate_red = row2['sig_str_hit_rate_red']
      imp_ctrled_time_ratio_red = row2['ctrl_time_ratio_red']
    else:
      print("Something Wrong Here 1")
    df1i.loc[index, 'imp_all_str_hit_ratio_red'] = imp_all_str_hit_ratio_red
    df1i.loc[index, 'imp_all_str_attempt_rate_red'] = imp_all_str_attempt_rate_red
    df1i.loc[index, 'imp_sig_str_hit_ratio_red'] = imp_sig_str_hit_ratio_red
    df1i.loc[index, 'imp_sig_str_attempt_rate_red'] = imp_sig_str_attempt_rate_red
    df1i.loc[index, 'imp_td_success_ratio_red'] = imp_td_success_ratio_red
    df1i.loc[index, 'imp_td_attempt_rate_red'] = imp_td_attempt_rate_red
    df1i.loc[index, 'imp_ctrl_time_ratio_red'] = imp_ctrl_time_ratio_red
    df1i.loc[index, 'imp_prop_match_completed_red'] = imp_prop_match_completed_red
    # NEW RED FEATURES
    df1i.loc[index, 'imp_all_str_absorb_ratio_red'] = imp_all_str_absorb_ratio_red
    df1i.loc[index, 'imp_all_str_absorb_rate_red'] = imp_all_str_absorb_rate_red
    df1i.loc[index, 'imp_sig_str_absorb_ratio_red'] = imp_sig_str_absorb_ratio_red
    df1i.loc[index, 'imp_sig_str_absorb_rate_red'] = imp_sig_str_absorb_rate_red
    df1i.loc[index, 'imp_ctrled_time_ratio_red'] = imp_ctrled_time_ratio_red
    if num_wins_red > 0:
      imp_num_win_NU_Decision_red = 0
      imp_num_win_U_Decision_red = 0
      imp_num_win_KO_TKO_red = 0
      imp_num_win_Submission_red = 0
      for index2, row2 in red_win_df.iterrows():
        if row2['red_name'] == row['red_name']:
          imp_num_win_NU_Decision_red += row2['method_NU_Decision']
          imp_num_win_U_Decision_red += row2['method_U_Decision']
          imp_num_win_KO_TKO_red += row2['method_KO/TKO']
          imp_num_win_Submission_red += row2['method_Submission']
        elif row2['blue_name'] == row['red_name']:
          imp_num_win_NU_Decision_red += row2['method_NU_Decision']
          imp_num_win_U_Decision_red += row2['method_U_Decision']
          imp_num_win_KO_TKO_red += row2['method_KO/TKO']
          imp_num_win_Submission_red += row2['method_Submission']
        else:
          print("Something Wrong Here 2")
      df1i.loc[index, 'imp_num_win_NU_Decision_red'] = imp_num_win_NU_Decision_red / num_wins_red
      df1i.loc[index, 'imp_num_win_U_Decision_red'] = imp_num_win_U_Decision_red / num_wins_red
      df1i.loc[index, 'imp_num_win_KO/TKO_red'] = imp_num_win_KO_TKO_red / num_wins_red
      df1i.loc[index, 'imp_num_win_Submission_red'] = imp_num_win_Submission_red / num_wins_red
  # extract rows from prophetic dataframe relevant to blue_name
  blue_df = new_df11.loc[(new_df11['card_date'] < row['card_date']) & ((new_df11['blue_name'] == row['blue_name']) | (new_df11['red_name'] == row['blue_name']))]
  blue_win_df = blue_df.loc[((blue_df['blue_name'] == row['blue_name']) & (blue_df['blue_win'] == 1)) | ((blue_df['red_name'] == row['blue_name']) & (blue_df['red_win']) == 1)]
  # use extracted rows to impute mean features
  num_fights_blue = blue_df.shape[0]
  num_wins_blue = blue_win_df.shape[0]
  df1i.loc[index, 'num_fights_blue'] = num_fights_blue
  if num_fights_blue > 0:
    imp_all_str_hit_ratio_blue = 0
    imp_all_str_attempt_rate_blue = 0
    imp_sig_str_hit_ratio_blue = 0
    imp_sig_str_attempt_rate_blue = 0
    imp_td_success_ratio_blue = 0
    imp_td_attempt_rate_blue = 0
    imp_ctrl_time_ratio_blue = 0
    imp_prop_match_completed_blue = 0
    # NEW BLUE FEATURES
    imp_all_str_absorb_ratio_blue = 0
    imp_all_str_absorb_rate_blue = 0
    imp_sig_str_absorb_ratio_blue = 0
    imp_sig_str_absorb_rate_blue = 0
    imp_ctrled_time_ratio_blue = 0
    df1i.loc[index, 'prop_wins_blue'] = num_wins_blue / num_fights_blue
    row2 = blue_df.loc[blue_df['card_date'] == blue_df['card_date'].max()].iloc[0]
    if row2['blue_name'] == row['blue_name']:
      # FORGOT TO ADD THIS IN
      imp_prop_match_completed_blue += row2['prop_match_completed']
      # END OF FORGOT TO ADD THIS IN 
      imp_all_str_hit_ratio_blue += row2['all_str_hit_ratio_blue']
      imp_all_str_attempt_rate_blue += row2['all_str_attempt_rate_blue']
      imp_sig_str_hit_ratio_blue += row2['sig_str_hit_ratio_blue']
      imp_sig_str_attempt_rate_blue += row2['sig_str_attempt_rate_blue']
      imp_td_success_ratio_blue += row2['td_success_ratio_blue']
      imp_td_attempt_rate_blue += row2['td_attempt_rate_blue']
      imp_ctrl_time_ratio_blue += row2['ctrl_time_ratio_blue']
      # NEW BLUE FEATURES
      imp_all_str_absorb_ratio_blue = row2['all_str_hit_ratio_red']
      imp_all_str_absorb_rate_blue = row2['all_str_hit_rate_red']
      imp_sig_str_absorb_ratio_blue = row2['sig_str_hit_ratio_red']
      imp_sig_str_absorb_rate_blue = row2['sig_str_hit_rate_red']
      imp_ctrled_time_ratio_blue = row2['ctrl_time_ratio_red']
    elif row2['red_name'] == row['blue_name']:
      imp_all_str_hit_ratio_blue += row2['all_str_hit_ratio_red']
      imp_all_str_attempt_rate_blue += row2['all_str_attempt_rate_red']
      imp_sig_str_hit_ratio_blue += row2['sig_str_hit_ratio_red']
      imp_sig_str_attempt_rate_blue += row2['sig_str_attempt_rate_red']
      imp_td_success_ratio_blue += row2['td_success_ratio_red']
      imp_td_attempt_rate_blue += row2['td_attempt_rate_red']
      imp_ctrl_time_ratio_blue += row2['ctrl_time_ratio_red']
      # NEW BLUE FEATURES
      imp_all_str_absorb_ratio_blue = row2['all_str_hit_ratio_blue']
      imp_all_str_absorb_rate_blue = row2['all_str_hit_rate_blue']
      imp_sig_str_absorb_ratio_blue = row2['sig_str_hit_ratio_blue']
      imp_sig_str_absorb_rate_blue = row2['sig_str_hit_rate_blue']
      imp_ctrled_time_ratio_blue = row2['ctrl_time_ratio_blue']
    else:
      print("Something Wrong Here 3")
    df1i.loc[index, 'imp_all_str_hit_ratio_blue'] = imp_all_str_hit_ratio_blue
    df1i.loc[index, 'imp_all_str_attempt_rate_blue'] = imp_all_str_attempt_rate_blue
    df1i.loc[index, 'imp_sig_str_hit_ratio_blue'] = imp_sig_str_hit_ratio_blue
    df1i.loc[index, 'imp_sig_str_attempt_rate_blue'] = imp_sig_str_attempt_rate_blue
    df1i.loc[index, 'imp_td_success_ratio_blue'] = imp_td_success_ratio_blue
    df1i.loc[index, 'imp_td_attempt_rate_blue'] = imp_td_attempt_rate_blue
    df1i.loc[index, 'imp_ctrl_time_ratio_blue'] = imp_ctrl_time_ratio_blue
    df1i.loc[index, 'imp_prop_match_completed_blue'] = imp_prop_match_completed_blue
    # NEW BLUE FEATURES
    df1i.loc[index, 'imp_all_str_absorb_ratio_blue'] = imp_all_str_absorb_ratio_blue
    df1i.loc[index, 'imp_all_str_absorb_rate_blue'] = imp_all_str_absorb_rate_blue
    df1i.loc[index, 'imp_sig_str_absorb_ratio_blue'] = imp_sig_str_absorb_ratio_blue
    df1i.loc[index, 'imp_sig_str_absorb_rate_blue'] = imp_sig_str_absorb_rate_blue
    df1i.loc[index, 'imp_ctrled_time_ratio_blue'] = imp_ctrled_time_ratio_blue
    if num_wins_blue > 0:
      imp_num_win_NU_Decision_blue = 0
      imp_num_win_U_Decision_blue = 0
      imp_num_win_KO_TKO_blue = 0
      imp_num_win_Submission_blue = 0
      for index2, row2 in blue_win_df.iterrows():
        if row2['blue_name'] == row['blue_name']:
          imp_num_win_NU_Decision_blue += row2['method_NU_Decision']
          imp_num_win_U_Decision_blue += row2['method_U_Decision']
          imp_num_win_KO_TKO_blue += row2['method_KO/TKO']
          imp_num_win_Submission_blue += row2['method_Submission']
        elif row2['red_name'] == row['blue_name']:
          imp_num_win_NU_Decision_blue += row2['method_NU_Decision']
          imp_num_win_U_Decision_blue += row2['method_U_Decision']
          imp_num_win_KO_TKO_blue += row2['method_KO/TKO']
          imp_num_win_Submission_blue += row2['method_Submission']
        else:
          print("Something Wrong Here 4")
      df1i.loc[index, 'imp_num_win_NU_Decision_blue'] = imp_num_win_NU_Decision_blue / num_wins_blue
      df1i.loc[index, 'imp_num_win_U_Decision_blue'] = imp_num_win_U_Decision_blue / num_wins_blue
      df1i.loc[index, 'imp_num_win_KO/TKO_blue'] = imp_num_win_KO_TKO_blue / num_wins_blue
      df1i.loc[index, 'imp_num_win_Submission_blue'] = imp_num_win_Submission_blue / num_wins_blue

In [9]:
# (part 2) export imputation engineered dataset (before cleaning) as csv file
path = "/content/drive/My Drive/Colab Notebooks/UFCScraper/custom_eng_last_fight_statistics_1.csv"
df1i.to_csv(path, index=False)

In [ ]:
from sklearn.model_selection import cross_val_score
# SECTION H2: FINAL EVALUATION OF LAST FIGHT IMPUTATION STRATEGY
# drop bouts where either red_fighter or blue_fighter is a newb
df1im = df1i[(df1i['num_fights_red'] >= 1) & (df1i['num_fights_blue'] >= 1)]
print("num total bouts:", len(df1i))
print("num rows after dropping", len(df1im))
# get some relevant statistics for analysis
print(df1im.columns)
print("Number Fights Red Stats:", df1im['num_fights_red'].describe())
print("Number Fights Blue Stats:", df1im['num_fights_blue'].describe())
# conduct logistic regression with history features
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['num_fights_red', 'prop_wins_red', 'imp_prop_match_completed_red',
                       'num_fights_blue', 'prop_wins_blue', 'imp_prop_match_completed_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_train = train_df1im['red_win']
X_test = test_df1im[['num_fights_red', 'prop_wins_red', 'imp_prop_match_completed_red',
                     'num_fights_blue', 'prop_wins_blue', 'imp_prop_match_completed_blue']]
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_test = test_df1im['red_win']
model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Accuracy:", mean_cv)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Metrics (History Features):")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# conduct logistic regression with offensive features
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                       'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                       'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                       'imp_ctrl_time_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                     'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                     'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                     'imp_ctrl_time_ratio_blue']]
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_test = test_df1im['red_win']
model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Accuracy:", mean_cv)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Metrics (Offensive Features):")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
y_train = y_train.reset_index(drop=True)
sm_model = sm.Logit(y_train, sm.add_constant(X_train_scaled)).fit(disp=0)
print(sm_model.pvalues)
print()
# conduct logistic regression with defensive features
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                       'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue',
                       'imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                       'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                       'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                       'imp_ctrl_time_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                     'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue',
                     'imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                     'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                     'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                     'imp_ctrl_time_ratio_blue']]
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_test = test_df1im['red_win']
model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Accuracy:", mean_cv)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Metrics (Defensive Features):")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
y_train = y_train.reset_index(drop=True)
sm_model = sm.Logit(y_train, sm.add_constant(X_train_scaled)).fit(disp=0)
print(sm_model.pvalues)
print()
# construct logistic regression with off + def features
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                       'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                     'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue']]
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_test = test_df1im['red_win']
model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Accuracy:", mean_cv)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Metrics (Defensive + Offensive Features):")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
y_train = y_train.reset_index(drop=True)
sm_model = sm.Logit(y_train, sm.add_constant(X_train_scaled)).fit(disp=0)
print(sm_model.pvalues)
print()
# construct logistic regression with off + def + history features
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['num_fights_red', 'prop_wins_red', 'imp_prop_match_completed_red',
                       'num_fights_blue', 'prop_wins_blue', 'imp_prop_match_completed_blue',
                       'imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                       'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue',
                       'imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                       'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                       'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                       'imp_ctrl_time_ratio_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
y_train = train_df1im['red_win']
X_test = test_df1im[['num_fights_red', 'prop_wins_red', 'imp_prop_match_completed_red',
                     'num_fights_blue', 'prop_wins_blue', 'imp_prop_match_completed_blue',
                     'imp_sig_str_absorb_ratio_red', 'imp_sig_str_absorb_rate_red', 'imp_ctrled_time_ratio_red',
                     'imp_sig_str_absorb_ratio_blue', 'imp_sig_str_absorb_rate_blue', 'imp_ctrled_time_ratio_blue',
                     'imp_sig_str_hit_ratio_red', 'imp_sig_str_attempt_rate_red', 'imp_td_success_ratio_red',
                     'imp_td_attempt_rate_red', 'imp_ctrl_time_ratio_red', 'imp_sig_str_hit_ratio_blue',
                     'imp_sig_str_attempt_rate_blue', 'imp_td_success_ratio_blue', 'imp_td_attempt_rate_blue',
                     'imp_ctrl_time_ratio_blue']]
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
y_test = test_df1im['red_win']
model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Accuracy:", mean_cv)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Metrics (Defensive + Offensive + History Features):")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
y_train = y_train.reset_index(drop=True)
sm_model = sm.Logit(y_train, sm.add_constant(X_train_scaled)).fit(disp=0)
print(sm_model.pvalues)
print()

In [ ]:
# SECTION H: TRAIN MODEL AND TEST USING IMPUTED DATA
# drop bouts where either red_fighter or blue_fighter is a newb
df1im = df1i[(df1i['num_fights_red'] >= 1) & (df1i['num_fights_blue'] >= 1)]
print("num total bouts:", len(df1i))
print("num rows after dropping", len(df1im))
print()
# predict which variables will be the most useful for predicting match result
X = df1im.drop(columns=['red_name', 'blue_name', 'red_win', 'blue_win', 'card_date'])
y = df1im['red_win']
selector = SelectKBest(score_func=mutual_info_classif, k=10)
selector.fit_transform(X, y)
scores = selector.scores_
ranked_features = X.columns[np.argsort(-scores)]
print("Feature ranking:")
for i, feature in enumerate(ranked_features):
    print("%d. %s (%f)" % (i+1, feature, scores[X.columns.get_loc(feature)]))
print()
# metrics when using 4 striking only features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking only with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking only + 4 strike defense only features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                      'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking + 4 defense only with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking + 4 strike defense + 2 control features with prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking only features + prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking only features + prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 6 custom selected features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 custom selected features with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 6 custom selected features + prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 custom selected features + prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()

num total bouts: 6716
num rows after dropping 4943

Feature ranking:
1. imp_ctrled_time_ratio_blue (0.012295)
2. imp_ctrl_time_ratio_red (0.011094)
3. imp_sig_str_attempt_rate_blue (0.010445)
4. imp_sig_str_hit_ratio_red (0.008491)
5. imp_ctrl_time_ratio_blue (0.008322)
6. imp_all_str_absorb_ratio_red (0.007821)
7. imp_num_win_U_Decision_blue (0.007358)
8. imp_all_str_attempt_rate_red (0.007288)
9. imp_sig_str_absorb_rate_red (0.006210)
10. imp_num_win_NU_Decision_red (0.005365)
11. imp_prop_match_completed_blue (0.005075)
12. imp_num_win_U_Decision_red (0.004777)
13. imp_sig_str_hit_ratio_blue (0.004760)
14. imp_td_success_ratio_red (0.003673)
15. imp_td_attempt_rate_red (0.003591)
16. imp_all_str_hit_ratio_blue (0.003584)
17. imp_all_str_absorb_ratio_blue (0.002964)
18. imp_sig_str_absorb_rate_blue (0.002826)
19. imp_num_win_Submission_blue (0.002724)
20. num_fights_red (0.001582)
21. num_fights_blue (0.001581)
22. imp_all_str_attempt_rate_blue (0.001513)
23. imp_all_str_absorb_rate_

In [ ]:
# TEST ON SOME DIFFERENT ML MODELS
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
# testing with logistic regression
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Logistic Regression")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# testing with support vector machine
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = svm.SVC()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Support Vector Machine")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# testing with random forests
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Random Forests")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()

Training Accuracy Logistic Regression
Accuracy: 0.6347994046952264
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.6319514661274014
Precision: 0.6376496191512514
Recall: 0.6319514661274014

Training Accuracy Support Vector Machine
Accuracy: 0.6279713229528398
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.6329625884732053
Precision: 0.6368366285119667
Recall: 0.6329625884732053

Training Accuracy Random Forests
Accuracy: 0.6135566259661701
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.60262891809909
Precision: 0.6221692491060786
Recall: 0.60262891809909



In [ ]:
# SECTION H (PART 2)
# drop bouts where either red_fighter or blue_fighter is a newb
df1im = df1i[(df1i['num_fights_red'] >= 1) & (df1i['num_fights_blue'] >= 1)]
print("num total bouts:", len(df1i))
print("num rows after dropping", len(df1im))
print()
# predict which variables will be the most useful for predicting match result
X = df1im.drop(columns=['red_name', 'blue_name', 'red_win', 'blue_win', 'card_date'])
y = df1im['red_win']
selector = SelectKBest(score_func=mutual_info_classif, k=10)
selector.fit_transform(X, y)
scores = selector.scores_
ranked_features = X.columns[np.argsort(-scores)]
print("Feature ranking:")
for i, feature in enumerate(ranked_features):
    print("%d. %s (%f)" % (i+1, feature, scores[X.columns.get_loc(feature)]))
print()
# metrics when using 4 striking only features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking only with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking only + 4 strike defense only features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                      'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking + 4 defense only with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking + 4 strike defense + 2 control features with prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 4 striking only features + prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (4 striking only features + prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 6 custom selected features with no prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 custom selected features with no prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# metrics when using 6 custom selected features + prev win stats
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                      'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics (6 custom selected features + prev win stats)")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()

num total bouts: 6716
num rows after dropping 4943

Feature ranking:
1. imp_sig_str_absorb_rate_red (0.012995)
2. imp_sig_str_hit_ratio_red (0.011182)
3. imp_ctrled_time_ratio_blue (0.010871)
4. imp_num_win_U_Decision_blue (0.010800)
5. imp_all_str_absorb_rate_blue (0.010072)
6. imp_num_win_NU_Decision_blue (0.008868)
7. imp_td_success_ratio_red (0.008622)
8. imp_all_str_absorb_rate_red (0.008367)
9. imp_num_win_NU_Decision_red (0.007913)
10. imp_num_win_U_Decision_red (0.007223)
11. imp_prop_match_completed_blue (0.006721)
12. imp_all_str_hit_ratio_red (0.006482)
13. imp_all_str_hit_ratio_blue (0.006052)
14. imp_sig_str_attempt_rate_blue (0.004904)
15. imp_num_win_Submission_red (0.004548)
16. imp_all_str_attempt_rate_red (0.004493)
17. imp_td_attempt_rate_blue (0.004424)
18. imp_td_attempt_rate_red (0.004056)
19. imp_all_str_absorb_ratio_red (0.003139)
20. imp_sig_str_absorb_ratio_blue (0.002807)
21. num_fights_blue (0.002804)
22. imp_ctrl_time_ratio_blue (0.002400)
23. imp_sig_str_h

In [ ]:
# TEST ON SOME DIFFERENT ML MODELS
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
# testing with logistic regression
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Logistic Regression")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# testing with support vector machine
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = svm.SVC()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Support Vector Machine")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()
# testing with random forests
train_df1im, test_df1im = train_test_split(df1im, test_size=0.2)
X_train = train_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
y_train = train_df1im['red_win']
X_test = test_df1im[['imp_all_str_attempt_rate_blue', 'imp_all_str_attempt_rate_red', 
                      'imp_all_str_hit_ratio_blue', 'imp_all_str_hit_ratio_red',
                      'imp_all_str_absorb_rate_blue', 'imp_all_str_absorb_rate_red',
                       'imp_all_str_absorb_ratio_blue', 'imp_all_str_absorb_ratio_red',
                       'imp_ctrl_time_ratio_blue', 'imp_ctrl_time_ratio_red',
                       'num_fights_red', 'num_fights_blue', 'prop_wins_red', 'prop_wins_blue']]
scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)
y_test = test_df1im['red_win']
model = model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring="accuracy")
mean_cv = cv_scores.mean()
print("Training Accuracy Random Forests")
print("Accuracy:", mean_cv)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = accuracy_score(y_test, y_pred)
print("Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print()

Training Accuracy Logistic Regression
Accuracy: 0.634801004976876
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.6127401415571284
Precision: 0.6177730192719486
Recall: 0.6127401415571284

Training Accuracy Support Vector Machine
Accuracy: 0.6284776520667638
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.6370070778564206
Precision: 0.6339937434827946
Recall: 0.6370070778564206

Training Accuracy Random Forests
Accuracy: 0.6186119156971628
Metrics 4 strike + 4 strike defense + 2 ctrl + 4 history
Accuracy: 0.6309403437815976
Precision: 0.6459770114942529
Recall: 0.6309403437815976

